In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Path: notebook.ipynb

In [19]:
df = pd.read_csv('data/data.csv')
df.head()

,id,name,artist_id,artist,playlist_id,playlist,genres,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,key,mode,time_signature
0,2zYzyRzz6pRmhPzyfMEC8s,Highway to Hell,711MCceyCBcFnzjGY4Q7Un,AC/DC,37i9dQZF1DWXRqgorJj26U,Rock Classics,"australian rock,hard rock,rock",0.574,0.913,-4.793,0.1330,0.061000,0.001580,0.1560,0.423,115.728,208400,6,0,4
1,1UBQ5GK8JaQjm5VbkBZY66,Sharp Dressed Man - 2008 Remaster,2AM4ilv6UzW0uMRuqKtDgN,ZZ Top,37i9dQZF1DWXRqgorJj26U,Rock Classics,"album rock,blues rock,classic rock,hard rock,rock",0.601,0.859,-5.263,0.0276,0.000359,0.001240,0.0871,0.446,125.145,258027,5,1,4
2,57JVGBtBLCfHw2muk5416J,Another One Bites The Dust - Remastered 2011,1dfeR4HaWDbWqFHLkxsg1d,Queen,37i9dQZF1DWXRqgorJj26U,Rock Classics,"classic rock,glam rock,rock",0.933,0.528,-6.472,0.1620,0.112000,0.329000,0.1630,0.756,109.975,214653,5,0,4
3,6pPWRBubXOBAHnjl5ZIujB,The Best,1zuJe6b1roixEKMOtyrEak,Tina Turner,37i9dQZF1DWXRqgorJj26U,Rock Classics,soft rock,0.662,0.737,-9.540,0.0296,0.108000,0.019100,0.1190,0.744,103.867,329800,0,1,4
4,40riOy7x9W7GXjyGp4pjAv,Hotel California - 2013 Remaster,0ECwFtbIWEVNwjlrfc6xoL,Eagles,37i9dQZF1DWXRqgorJj26U,Rock Classics,"album rock,classic rock,heartland rock,mellow ...",0.579,0.508,-9.484,0.0270,0.005740,0.000494,0.0575,0.609,147.125,391376,2,1,4


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2921 entries, 0 to 2920
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                2921 non-null   object 
 1   name              2921 non-null   object 
 2   artist_id         2921 non-null   object 
 3   artist            2921 non-null   object 
 4   playlist_id       2921 non-null   object 
 5   playlist          2921 non-null   object 
 6   genres            2470 non-null   object 
 7   danceability      2921 non-null   float64
 8   energy            2921 non-null   float64
 9   loudness          2921 non-null   float64
 10  speechiness       2921 non-null   float64
 11  acousticness      2921 non-null   float64
 12  instrumentalness  2921 non-null   float64
 13  liveness          2921 non-null   float64
 14  valence           2921 non-null   float64
 15  tempo             2921 non-null   float64
 16  duration_ms       2921 non-null   int64  


In [21]:
# see if duplicates exist

df[df.duplicated()]


,id,name,artist_id,artist,playlist_id,playlist,genres,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,key,mode,time_signature
1141,1qwQ1P3FwFouG1zTIBkGXU,Anlamazdın,74VZuXJwRTaBExjMhYBYi4,Ayla Dikmen,37i9dQZF1DXaAWn8mryoAL,Türkçe 70'ler,classic turkish pop,0.324,0.573,-7.224,0.0495,0.7510,0.001270,0.548,0.433,116.735,222347,11,0,4
1142,4Mye0IMIFvnBWxrZGJ4L1t,Ara Sıra Bazı Bazı,3iVIrcJmrV7GawrxVWsBUF,Nilüfer,37i9dQZF1DXaAWn8mryoAL,Türkçe 70'ler,turkish pop,0.488,0.457,-10.504,0.0875,0.4950,0.000000,0.230,0.578,85.171,170063,7,1,4
1145,294JL3Xexu3sVRH9tpn60G,Aşkın Gözü Kör Mü,5L7Hl11e6LAeNWfvs5siH4,Rana Alagöz,37i9dQZF1DXaAWn8mryoAL,Türkçe 70'ler,"anadolu rock,classic turkish pop,turkish psych",0.477,0.789,-5.426,0.0699,0.5370,0.000000,0.325,0.856,87.798,210560,4,0,4
1146,3S7VCuq7pxnmr8S3UmDsQn,Olmaz Bu İş Olamaz,5vuQpKX1QG7WlQ40C27HfD,Semiramis Pekkan,37i9dQZF1DXaAWn8mryoAL,Türkçe 70'ler,classic turkish pop,0.700,0.851,-7.742,0.0657,0.8070,0.000007,0.370,0.915,122.099,174124,7,1,4
1147,4h5vixuae5dCWrSYd88trW,Olmaz Olmaz Deme,7xKyfYFRIHqXHrQh53yaug,Nil Burak,37i9dQZF1DXaAWn8mryoAL,Türkçe 70'ler,classic turkish pop,0.566,0.637,-11.007,0.1100,0.0571,0.000000,0.736,0.670,97.553,219987,11,1,4


In [23]:
# drop duplicates

df.drop_duplicates(inplace=True)
df[df.duplicated()]

,id,name,artist_id,artist,playlist_id,playlist,genres,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,key,mode,time_signature
